In [ ]:
import glob
import os
import numpy as np
import pandas as pd


In [ ]:
path = "" #path to master folder containing individual mice folders with DLC-generated tracking coordinates for individual reach trials 

for folder in path:                 #Intiation for each mouse                                                                                                                                                                                                                     
    
    print(folder)
    
    csv_files = glob.glob(os.path.join(folder, "*.csv"))

    MeanPortBodyX = []
    MeanPortBodyY = []
    
    PopPalmEndPointX = []
    PopPalmEndPointY = []

    PopPalmStartPointX = [] 
    PopPalmStartPointY = []

    Slopes = []

    sortedtimeList = []

    filenp = 0
    filep = 0


    for file in csv_files:          #Initiation for each trial
        
        #print(file)
        
        reachFile = pd.read_csv(file, header = 1 )

        palmCoords = []

        PalmX = reachFile["LeftPalm"]
        PalmX = PalmX.tail(-1)
        PalmY = reachFile["LeftPalm.1"]
        PalmY = PalmY.tail(-1)
        PalmCoordsL = reachFile["LeftPalm.2"]
        PalmCoordsL = PalmCoordsL.tail(-1)

        PortBodyX = reachFile["PortBody"]
        PortBodyX = PortBodyX.tail(-1)
        PortBodyX = PortBodyX.astype(float)

        PortBodyY = reachFile["PortBody.1"]
        PortBodyY = PortBodyY.tail(-1)    
        PortBodyY = PortBodyY.astype(float)



        start = 0
        attempt = 0

        # reach tracking
        try:
            for i in range(1, len(PalmX)):  
                
                if ((float(PalmCoordsL[i]) >= 0.70)  and (300 <= float(PalmX[i])<= 500)  and (start == 0) and (attempt==0)):      
                    if all(float(PalmCoordsL[i+j]) >= 0.70 for j in range(1, 6)) and all(float(PalmX[i+n])- float(PalmX[i+n-1]) < 0  for n in range(1,6)):
                        palmCoords.append((float(PalmX[i]), float(PalmY[i])))
                        attemptlen +=1
                        start +=1
                        startPoint = i


                #outward movement direction is along inverse x axis

                elif   (100 <= float(PalmX[i]) <= 500)  and (start>0) and (attempt == 0):
                    if all((float(PalmX[i+j])-float(PalmX[i]) > 0 for j in range(-3, 0))) and all((float(PalmX[i+j])-float(PalmX[i]) > 0 for j in range(1, 3))) and np.min(palmCoords)< 300 and (i-startPoint < 13):
                        palmCoords.append((float(PalmX[i]), float(PalmY[i])))
                        attempt = attempt+1
                        
                                            
                    elif (float(PalmX[i+1])-float(PalmX[i]) < 0) and (i-startPoint < 13):
                        palmCoords.append((float(PalmX[i]), float(PalmY[i])))
                        attemptlen +=1

                    elif np.min(palmCoords)>300 and (i-startPoint > 13):
                        raise Exception
                        
                    
                elif (attempt>0):
                    break

       
            palmCoords = np.array(palmCoords)
            palmCX, palmCY = palmCoords.T

            dx = palmCX[1:]-palmCX[:-1]
            dy = palmCY[1:]-palmCY[:-1]

            vels = np.sqrt(dx**2+dy**2)
            
            timeList = np.linspace(0, len(palmCX)/150, len(palmCX))


            sortedPalmX, sortedPalmY, sortedTime = zip(*sorted(zip( palmCX,palmCY, timeList)))
            sortedtimeList.append(sortedTime)
            
            PopPalmEndPointX.append(sortedPalmX[0])
            PopPalmEndPointY.append(sortedPalmY[0])

            PopPalmStartPointX.append(sortedPalmX[-1])
            PopPalmStartPointY.append(sortedPalmY[-1])

            MeanPortBodyX.append(np.mean(PortBodyX))
            MeanPortBodyY.append(np.mean(PortBodyY))

            filenp = filenp +1
        except Exception as error:
            #print(error)
            print(file.lower())
            filep +=1
            continue

    print("filenp",filenp)
    print("filep", filep)



    
    #for i in ((list((-((PopPalmEndPointY-np.mean(MeanPortBodyY))))))):
        #print(i)

    #print("\n")

    #for x in ( (list((-((PopPalmStartPointY-np.mean(MeanPortBodyY))))))):
       #print (x)

    #print("\n")

    #for s in (list(np.arctan((abs((PopPalmStartPointY)-(PopPalmEndPointY)))/(abs((PopPalmStartPointX)-(PopPalmEndPointX))))))
        #print(s)


    
    #print((np.std((PopPalmEndPointY-np.mean(MeanPortBodyY)))))
    #print((np.std((PopPalmStartPointY-np.mean(MeanPortBodyY)))))    
    

 
